In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi
# fugi = reload(fugi)

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["png"]

import re

#for the cartesian product
import itertools

#to smooth the MFPADs
import scipy as sp
import scipy.ndimage

#to create the mesh
import triangulation as tr
from scipy.spatial import Delaunay
from scipy.ndimage.filters import gaussian_filter
from pyntcloud import PyntCloud, structures

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

# ##550eV - 11.5eV CH9
# #update nov Kilian binning (36,18)
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH9_550eV_CR_Knov.root");en="550eV";CH="CH9";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH9_550eV_CL_Knov.root")
# fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH9_550eV_CR_Knov.root")
# fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH9_550eV_CL_Knov.root")

# #546eV - 8.2eV CH9
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CR_9600-3700ns_multiCH9.root");en="546eV";CH="CH9";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CL_9600-3700ns_multiCH9.root")

# # 544eV - 6.1eV CH9
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CR_9600-3700ns_multiCH9.root");en="544eV";CH="CH9";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CL_9600-3700ns_multiCH9.root")

# # 542eV - 4eV CH9
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CR_9600-3700ns_COMBI_multiCH9.root");en="542eV";CH="CH9";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CL_9600-3700ns_COMBI_multiCH9.root")

#541eV 3eV CH9
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CR_9600-3700ns_multiCH9.root");en="541eV";CH="CH9";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CL_9600-3700ns_multiCH9.root")

fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CR_Knov_test.root");en="550eV";CH="CH12";
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CL_Knov_test.root")
fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov_test.root")
fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CL_Knov_test.root")

#uncomment this for the special case!
CH="CH12"
path = "angular_distr_el/"+CH+"/"

#move first the ID you need: ID1 IS JUST FOR CH12
IDs = ["ID_ALL_mol_e0_valid/EN_gate/","ID1_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/", "ID4_mol_e0_valid/EN_gate/"]
dirs = ["MFPADs_multinew/","MFPADs_multinew_phi/"]

loc=path+IDs[1]+dirs[0]
ID=str(IDs[1]).split("_")[0]

noprint = True

#to alow the helicity comparison without the S enantiomer
if en != "550eV":
    fileSCR = None
    fileSCL = None

#new uproot4
# https://uproot.readthedocs.io/en/latest/basic.html
d=dict(fileRCR[loc].classnames()) # shows the type of each item
# k=fileRCR[loc].keys() #same as .classnames()
check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
d1=type(check)
check.all_members; #all members of the single histogram

#set the gaussian for the smoothing before PECD and/or contrast maps
gaussian=0.3

In [ ]:
#Loading of the MFPADS and cos len = 27
MFPAD_RCR,ctheta_RCR,ctheta_RCR_err,cosphi_photon,MFPAD_axis,ctheta_axis,ctheta_axis_red=fugi.import_MFPAD(fileRCR, loc, full=True)
MFPAD_RCL,ctheta_RCL,ctheta_RCL_err=fugi.import_MFPAD(fileRCL, loc)
#projection of cos on x axis: len = 18
cosx_RCR=fugi.projection(MFPAD_RCR,alongaxis=0)
cosx_RCL=fugi.projection(MFPAD_RCL,alongaxis=0)

if fileSCL and fileSCR is not None:    
    MFPAD_SCR,ctheta_SCR,ctheta_SCR_err=fugi.import_MFPAD(fileSCR, loc)
    MFPAD_SCL,ctheta_SCL,ctheta_SCL_err=fugi.import_MFPAD(fileSCL, loc)
    #projection of cos on x axis
    cosx_SCR=fugi.projection(MFPAD_SCR,alongaxis=0)
    cosx_SCL=fugi.projection(MFPAD_SCL,alongaxis=0)

In [ ]:
#import 3d graph kilian
MFPAD_RCLtest=fugi.import_MFPAD3D(fileRCL, loc)
MFPAD_RCRtest=fugi.import_MFPAD3D(fileRCR, loc)
print("MFPAD_RCLtest shape ",MFPAD_RCLtest.shape)

#sum
MFPAD_RCLtest_sum=np.sum(MFPAD_RCLtest,axis=0)
MFPAD_RCRtest_sum=np.sum(MFPAD_RCRtest,axis=0)

#normalise each and sum
# MFPAD_RCLtest_sum=np.sum(fugi.normalise_matrix(MFPAD_RCLtest),axis=0)
# MFPAD_RCRtest_sum=np.sum(fugi.normalise_matrix(MFPAD_RCRtest),axis=0)
MFPAD_Rtest_sum = np.add(MFPAD_RCLtest_sum,MFPAD_RCRtest_sum)

if fileSCL and fileSCR is not None:
    MFPAD_SCRtest=fugi.import_MFPAD3D(fileSCR, loc)
    MFPAD_SCLtest=fugi.import_MFPAD3D(fileSCL, loc)
    print("MFPAD_SCLtest shape ",MFPAD_SCLtest.shape)
    MFPAD_SCRtest_sum=np.sum(MFPAD_SCRtest,axis=0)
    MFPAD_SCLtest_sum=np.sum(MFPAD_SCLtest,axis=0)

    # MFPAD_SCLtest_sum=np.sum(fugi.normalise_matrix(MFPAD_SCLtest),axis=0)
    # MFPAD_SCRtest_sum=np.sum(fugi.normalise_matrix(MFPAD_SCRtest),axis=0)

    #flip along cos(theta): JUST ALONG COS(theta) -> axis=1
    MFPAD_SCLtest_flip_sum=np.flip(MFPAD_SCLtest_sum,axis=1)
    MFPAD_SCRtest_flip_sum=np.flip(MFPAD_SCRtest_sum,axis=1)
    MFPAD_Stest_sum = np.add(MFPAD_SCLtest_flip_sum,MFPAD_SCRtest_flip_sum)

    MFPAD_tottest=np.add(MFPAD_Stest_sum,MFPAD_Rtest_sum)

    print("tottes shape ",MFPAD_tottest.shape)
else:
    MFPAD_tottest=MFPAD_Rtest_sum

In [ ]:
##########################b1 map x,y coordiantes###############################
"""
PHI - CTHETA photon with respect to the LF (in sperical coordinates)
NOTE the go coordiantes are organised accordinf to the SORTED MFPAD, therefore cos(theta)=[-1,1]
REDUCED=TRUE => ADJUSTED TO EXPERIMENT: -0.835,0.835,-165,165
REDUCED=FALSE => ADJUSTED TO THEORY: -1,1,-180,180
"""
# fugi = reload(fugi)

xgo_phi,ygo_phi,cosphi_adj_phi = fugi.create_gocoords(a=0, cos_lin=True, source=True)
xgo_cos,ygo_cos,cosphi_adj_cos = fugi.create_gocoords(a=1, cos_lin=True, source=True)

# list(zip(xgo_phi,ygo_phi))
print("(xgo_cos (phi), ygo_cos (ctheta)) \n", list(zip(xgo_cos,ygo_cos))[:6])
print("\n cosphi_adj_cos  \n", cosphi_adj_cos[:6])
print("\n cosphi_adj_phi \n", cosphi_adj_phi[:6])

In [ ]:
#COORDIANTES PHOTON ROTATION
#adjusting cosphi with a shifting to have it equal to theory
ctheta_axis_red=ctheta_axis_red[0]; #Ffrom ccos(theta)

phiM=(MFPAD_axis[0][0][1:] + MFPAD_axis[0][0][:-1])/2 #phi = (36,), MFPAD_axis = (37,)
cosM=(MFPAD_axis[0][1][1:] + MFPAD_axis[0][1][:-1])/2 #cos = (18,), MFPAD_axis = (19,)

print("MFPAD_xy shape = ", MFPAD_axis[0][0].shape)
print("phiM shape = ", phiM.shape, ", phiM[0] = ",phiM[0])
print("cosM shape = ", cosM.shape, ", cosM[0] = ",cosM[0])

dfind=pd.DataFrame(cosphi_adj_phi, columns=["ctheta","phi"])

In [ ]:
#normalising entries, important just for PECD
MFPAD_RCR_norm = fugi.normalise_with_err(MFPAD_RCR) # z values normalise to number of event
MFPAD_RCL_norm = fugi.normalise_with_err(MFPAD_RCL) # z values normalise to number of event
cosx_RCR_norm = fugi.normalise_with_err(cosx_RCR) # z values normalise to number of event
cosx_RCL_norm = fugi.normalise_with_err(cosx_RCL) # z values normalise to number of event

if fileSCL and fileSCR is not None:
    MFPAD_SCR_norm = fugi.normalise_with_err(MFPAD_SCR) # z values normalise to number of event
    MFPAD_SCL_norm = fugi.normalise_with_err(MFPAD_SCL) # z values normalise to number of event
    cosx_SCR_norm = fugi.normalise_with_err(cosx_SCR) # z values normalise to number of event
    cosx_SCL_norm = fugi.normalise_with_err(cosx_SCL) # z values normalise to number of event

In [ ]:
#flipping; axis=1 flips along cos(theta), axis=0 along phi
MFPAD_RCL_flip=[]
MFPAD_RCR_flip=[]

#flipping sorted S enantiomer for R enantiomer: JUST ALONG COS(theta) -> axis=1
for el in MFPAD_RCL:
    MFPAD_RCL_flip.append(np.flip(el, axis=1))
MFPAD_RCL_flip=np.array(MFPAD_RCL_flip)

#flipping sorted S enantiomer for R enantiomer: JUST ALONG COS(theta) -> axis=1
for el in MFPAD_RCR:
    MFPAD_RCR_flip.append(np.flip(el, axis=1))
MFPAD_RCR_flip=np.array(MFPAD_RCR_flip)

if fileSCL and fileSCR is not None:
    MFPAD_SCL_flip=[]
    MFPAD_SCR_flip=[]

    for el in MFPAD_SCL:
        MFPAD_SCL_flip.append(np.flip(el, axis=1))
    MFPAD_SCL_flip=np.array(MFPAD_SCL_flip)

    for el in MFPAD_SCR:
        MFPAD_SCR_flip.append(np.flip(el, axis=1))
    MFPAD_SCR_flip=np.array(MFPAD_SCR_flip)    

#test flip cos(theta) -> axis=1
fig,ax=plt.subplots(1,3)
ax[0].imshow(MFPAD_RCL[0].T)
ax[1].imshow(MFPAD_RCL_flip[0].T)
ax[2].imshow(MFPAD_RCL_flip_test[0].T)

In [ ]:
#sum within an experimemt axis=0 is among the 72 elements
#single enantiomer and helicity
#sum all MFPAD: NO NORMALIZATION
MFPAD_RCR_sum=np.sum(MFPAD_RCR, axis=0)
MFPAD_RCL_sum=np.sum(MFPAD_RCL, axis=0)
MFPAD_R_sum=np.add(MFPAD_RCR_sum,MFPAD_RCL_sum)

if fileSCL and fileSCR is not None:
    #single enantiomer and helicity
    MFPAD_SCR_sum=np.sum(MFPAD_SCR, axis=0)
    MFPAD_SCL_sum=np.sum(MFPAD_SCL, axis=0)
    MFPAD_SCR_flip_sum=np.sum(MFPAD_SCR_flip, axis=0)
    MFPAD_SCL_flip_sum=np.sum(MFPAD_SCL_flip, axis=0)
    MFPAD_S_sum=np.add(MFPAD_SCR_flip_sum,MFPAD_SCL_flip_sum)

    #sum of two enantiomers: there is no influence of the ellicity
    # MFPAD_tot_sum is a R ENANTIOMER
    MFPAD_tot_sum=np.add(MFPAD_R_sum,MFPAD_S_sum)

    #!NOTE it exists just for for 11.7 eV, and flips from R-enant to S-enant
    MFPAD_tot_sum_flip=np.flip(MFPAD_tot_sum,axis=1) #the cos of the single MFPAD has to be reversed: JUST ALONG COS(theta) -> axis=1

    # has to be normalized to avoid the contribution of unballanced counts
    MFPAD_tot_sum_norm=fugi.normalise_with_err(MFPAD_tot_sum, normtype=2)
    MFPAD_tot_sum_flip_norm=fugi.normalise_with_err(MFPAD_tot_sum_flip,normtype=2)

else:
    MFPAD_tot_sum=MFPAD_R_sum
    MFPAD_tot_sum_norm=fugi.normalise_with_err(MFPAD_tot_sum, normtype=2)

In [ ]:
print(MFPAD_RCR.shape)
print(MFPAD_RCR.max(),MFPAD_RCR.min())

In [ ]:
#sorting
#let´s work just with NOT NORMALISED becuase of MFPAD
MFPAD_RCR_cos=fugi.sorting_array(MFPAD_RCR, theory=False, a=1)
MFPAD_RCL_cos=fugi.sorting_array(MFPAD_RCL, theory=False, a=1)
MFPAD_RCR_phi=fugi.sorting_array(MFPAD_RCR, theory=False, a=2)
MFPAD_RCL_phi=fugi.sorting_array(MFPAD_RCL, theory=False, a=2)
MFPAD_RCR_phi_norm=fugi.normalise_with_err(MFPAD_RCR_phi,normtype=2)
MFPAD_RCL_phi_norm=fugi.normalise_with_err(MFPAD_RCL_phi,normtype=2)

MFPAD_RCL_flip_cos=fugi.sorting_array(MFPAD_RCL_flip, theory=False, a=1)
MFPAD_RCL_flip_phi=fugi.sorting_array(MFPAD_RCL_flip, theory=False, a=2)
MFPAD_RCR_flip_cos=fugi.sorting_array(MFPAD_RCR_flip, theory=False, a=1)
MFPAD_RCR_flip_phi=fugi.sorting_array(MFPAD_RCR_flip, theory=False, a=2)


if fileSCL and fileSCR is not None:
    MFPAD_SCR_cos=fugi.sorting_array(MFPAD_SCR, theory=False, a=1)
    MFPAD_SCL_cos=fugi.sorting_array(MFPAD_SCL, theory=False, a=1)
    MFPAD_SCR_phi=fugi.sorting_array(MFPAD_SCR, theory=False, a=2)
    MFPAD_SCL_phi=fugi.sorting_array(MFPAD_SCL, theory=False, a=2)
    MFPAD_SCR_phi_norm=fugi.normalise_with_err(MFPAD_SCR_phi,normtype=2)
    MFPAD_SCL_phi_norm=fugi.normalise_with_err(MFPAD_SCL_phi,normtype=2)

    MFPAD_SCL_flip_cos=fugi.sorting_array(MFPAD_SCL_flip, theory=False, a=1)
    MFPAD_SCL_flip_phi=fugi.sorting_array(MFPAD_SCL_flip, theory=False, a=2)
    MFPAD_SCR_flip_cos=fugi.sorting_array(MFPAD_SCR_flip, theory=False, a=1)
    MFPAD_SCR_flip_phi=fugi.sorting_array(MFPAD_SCR_flip, theory=False, a=2)

In [ ]:
#smart flipping for the single 72 MFPAD image. Has to be in phi because of the structure of reshape, do the cos just as a check
temp_RCLCR=np.empty([12,6,648])
temp_PECD_R=np.empty([12,6,648])
temp_PECD_R_flip=np.empty([12,6,648])
temp_PECD_R_sm=np.empty([12,6,648])
temp_cPECD_R=np.empty([12,6,18])
temp_cPECD_err_R=np.empty([12,6,18])
temp_cPECD_R_sm=np.empty([12,6,18])
temp_cPECD_err_R_sm=np.empty([12,6,18])

nphi=MFPAD_RCL.shape[1];ncos=MFPAD_RCL.shape[2]

#sum helicities: -cos(theta), phi+180 in photon coordiantes
for j in range(12):
    for i in range(6):
        if j>5:
            temp_RCLCR[j,i]=np.add(MFPAD_RCL_phi.reshape(12,6,-1)[j,i],MFPAD_RCR_phi.reshape(12,6,-1)[j-6,5-i])
        else:
            temp_RCLCR[j,i]=np.add(MFPAD_RCL_phi.reshape(12,6,-1)[j,i],MFPAD_RCR_phi.reshape(12,6,-1)[6+j,5-i])

MFPAD_RCLCR=np.array(temp_RCLCR).reshape(72,nphi,ncos)
MFPAD_RCLCR_sum=np.sum(MFPAD_RCLCR, axis=0)

#sorting
MFPAD_RCLCR_cos=fugi.sorting_array(MFPAD_RCLCR, theory=False, a=1)
MFPAD_RCLCR_phi=fugi.sorting_array(MFPAD_RCLCR, theory=False, a=2) #equal to the original!
MFPAD_RCLCR_phi_norm=fugi.normalise_with_err(MFPAD_RCLCR_phi)

temp=[]
for el in temp_RCLCR.reshape(72,nphi,ncos):
    temp.append(np.flip(el,axis=1)) #flip ctheta axis=1
temp_RCLCR_flip=np.array(temp).reshape(12,6,-1)

MFPAD_RCLCR_flip=np.array(temp_RCLCR_flip).reshape(72,nphi,ncos) #artificial S-enant

temp=[]
for el in MFPAD_RCLCR_phi_norm:
    temp.append(fugi.projection(el, alongaxis=0))
ctheta_R_phi_norm=np.array(temp).reshape(72,ncos)

for j in range(12):
    for i in range(6):
        if j>5:
            temp_PECD_R[j,i]=np.divide(np.subtract(MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[j,i],MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[j-6,5-i]),np.add(MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[j,i],MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[j-6,5-i]))
            temp_PECD_R_sm[j,i]=np.divide(np.subtract(gaussian_filter(MFPAD_RCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(MFPAD_RCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[j-6,5-i]),np.add(gaussian_filter(MFPAD_RCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(MFPAD_RCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[j-6,5-i]))
            temp_PECD_R_flip[j,i]=np.divide(np.subtract(MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[j-6,5-i],MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[j,i]),np.add(MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[j-6,5-i],MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[j,i]))
            temp_cPECD_R[j,i]=np.divide(np.subtract(ctheta_R_phi_norm.reshape(12,6,-1)[j,i],ctheta_R_phi_norm.reshape(12,6,-1)[j-6,5-i]),np.add(ctheta_R_phi_norm.reshape(12,6,-1)[j,i],ctheta_R_phi_norm.reshape(12,6,-1)[j-6,5-i]))
            temp_cPECD_err_R[j,i]=fugi.error_calc(ctheta_R_phi_norm.reshape(12,6,-1)[j,i],ctheta_R_phi_norm.reshape(12,6,-1)[j-6,5-i])
            temp_cPECD_R_sm[j,i]=np.divide(np.subtract(gaussian_filter(ctheta_R_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(ctheta_R_phi_norm,sigma=gaussian).reshape(12,6,-1)[j-6,5-i]),np.add(gaussian_filter(ctheta_R_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(ctheta_R_phi_norm,sigma=gaussian).reshape(12,6,-1)[j-6,5-i]))

        else:
            temp_PECD_R[j,i]=np.divide(np.subtract(MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[j,i],MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[6+j,5-i]),np.add(MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[j,i],MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[6+j,5-i]))
            temp_PECD_R_sm[j,i]=np.divide(np.subtract(gaussian_filter(MFPAD_RCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(MFPAD_RCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[6+j,5-i]),np.add(gaussian_filter(MFPAD_RCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(MFPAD_RCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[6+j,5-i]))
            temp_PECD_R_flip[j,i]=np.divide(np.subtract(MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[6+j,5-i],MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[j,i]),np.add(MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[6+j,5-i],MFPAD_RCLCR_phi_norm.reshape(12,6,-1)[j,i]))
            temp_cPECD_R[j,i]=np.divide(np.subtract(ctheta_R_phi_norm.reshape(12,6,-1)[j,i],ctheta_R_phi_norm.reshape(12,6,-1)[6+j,5-i]),np.add(ctheta_R_phi_norm.reshape(12,6,-1)[j,i],ctheta_R_phi_norm.reshape(12,6,-1)[6+j,5-i]))
            temp_cPECD_err_R[j,i]=fugi.error_calc(ctheta_R_phi_norm.reshape(12,6,-1)[j,i],ctheta_R_phi_norm.reshape(12,6,-1)[6+j,5-i])
            temp_cPECD_R_sm[j,i]=np.divide(np.subtract(gaussian_filter(ctheta_R_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(ctheta_R_phi_norm,sigma=gaussian).reshape(12,6,-1)[6+j,5-i]),np.add(gaussian_filter(ctheta_R_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(ctheta_R_phi_norm,sigma=gaussian).reshape(12,6,-1)[6+j,5-i]))

PECD_RCLCR=np.array(temp_PECD_R).reshape(72,nphi,ncos)
PECD_RCRCL=np.array(temp_PECD_R).reshape(72,nphi,ncos) #to be compared with MOx
PECD_RCLCR_smooth=np.array(temp_PECD_R_sm).reshape(72,nphi,ncos)

cPECD_norm_phi=np.array(temp_cPECD_R).reshape(72,-1) 
cPECD_norm_phi_sm=np.array(temp_cPECD_R_sm).reshape(72,-1)
errPECD_phi=np.array(temp_cPECD_err_R/100).reshape(72,-1) #works without dividing 100

if fileSCL and fileSCR is not None:
    temp_SCLCR=np.empty([12,6,648])
    temp_PECD_S=np.empty([12,6,648])
    temp_PECD_SR=np.empty([12,6,648])
    temp_PECD_SR_mod=np.empty([12,6,648])
    temp_PECD_SR_sm=np.empty([12,6,648])
    temp_PECD_RS=np.empty([12,6,648])
    temp_cPECD_RS=np.empty([12,6,18])
    temp_cPECD_RS_sm=np.empty([12,6,18])
    temp_cPECD_err_SR=np.empty([12,6,18])
    temp_totordered_S=np.empty([12,6,648])
    temp_totordered_R=np.empty([12,6,648]) #to compare to MOx
    temp_totordered_modMFPAD=np.empty([12,6,648])

    #sum helicities: -cos(theta), phi+180 in photon coordiantes
    for j in range(12):
        for i in range(6):
            if j>5:
                temp_SCLCR[j,i]=np.add(MFPAD_SCL_phi.reshape(12,6,-1)[j,i],MFPAD_SCR_phi.reshape(12,6,-1)[j-6,5-i])
            else:
                temp_SCLCR[j,i]=np.add(MFPAD_SCL_phi.reshape(12,6,-1)[j,i],MFPAD_SCR_phi.reshape(12,6,-1)[6+j,5-i])

    MFPAD_SCLCR=np.array(temp_SCLCR).reshape(72,nphi,ncos)
    MFPAD_SCLCR_sum=np.sum(MFPAD_SCLCR, axis=0)

    temp=[]
    for el in temp_SCLCR.reshape(72,nphi,ncos):
        temp.append(np.flip(el,axis=1)) #flip ctheta axis=1
    temp_SCLCR_flip=np.array(temp).reshape(12,6,-1)
    MFPAD_SCLCR_flip=np.array(temp_SCLCR_flip).reshape(72,nphi,ncos)

    #sorting
    MFPAD_SCLCR_cos=fugi.sorting_array(MFPAD_SCLCR, theory=False, a=1)
    MFPAD_SCLCR_phi=fugi.sorting_array(MFPAD_SCLCR, theory=False, a=2) #equal to the original!
    MFPAD_SCLCR_phi_flip=fugi.sorting_array(MFPAD_SCLCR_flip, theory=False, a=2) #equal to the original!
    MFPAD_SCLCR_phi_norm=fugi.normalise_with_err(MFPAD_SCLCR_phi,normtype=2)

    temp=[]
    for el in MFPAD_SCLCR_phi_norm:
        temp.append(fugi.projection(el, alongaxis=0))
    ctheta_S_phi_norm=np.array(temp).reshape(72,ncos)

    #sum enantiomer: -cos(theta) fliped axis=1 cos!, -cos(theta) photon coordinates. it shouldn´t be normalized!
    for j in range(12):
        for i in range(6):
            temp_totordered_S[j,i]=np.add(temp_SCLCR[j,i],gaussian_filter(temp_RCLCR_flip, sigma=0.3)[j,5-i]) #S-enant, the correct version to Kilian
            temp_totordered_modMFPAD[j,i]=np.add(temp_SCLCR[j,i],gaussian_filter(temp_RCLCR, sigma=0.3)[j,i]) #S-enant, better for 72 MFPADs and SR PECD? yes
            temp_totordered_R[j,i]=np.add(gaussian_filter(temp_RCLCR, sigma=0.5)[j,i],np.flip(temp_SCLCR[j,5-i].reshape(nphi,ncos),axis=1).reshape(-1)) #works, but R is noisy
            # temp_totordered_S[j,i]=np.add(temp_SCLCR[j,i],np.flip(gaussian_filter(temp_RCLCR, sigma=0.3)[j,5-i].reshape(nphi,ncos),axis=0).reshape(-1)) #the correct version to Kilian
            # temp_totordered_R[j,i]=np.add(gaussian_filter(temp_RCLCR, sigma=0.3)[j,i],temp_SCLCR_flip[j,5-i]) #the correct version to Kilian: doesn't work
            # temp_totordered_R[j,i]=np.flip(temp_totordered[j,5-i].reshape(nphi,ncos),axis=1).reshape(-1)

    MFPAD_totordered_S=np.array(temp_totordered_S).reshape(72,nphi,ncos)
    MFPAD_totordered_S_mod=np.array(temp_totordered_modMFPAD).reshape(72,nphi,ncos)
    MFPAD_totordered_R=np.array(temp_totordered_R).reshape(72,nphi,ncos)

    #sorting
    MFPAD_totordered_S_cos=fugi.sorting_array(MFPAD_totordered_S, theory=False, a=1)
    MFPAD_totordered_S_phi=fugi.sorting_array(MFPAD_totordered_S, theory=False, a=2) #equal to the original!
    MFPAD_totordered_S_mod_cos=fugi.sorting_array(MFPAD_totordered_S_mod, theory=False, a=1)
    MFPAD_totordered_S_mod_phi=fugi.sorting_array(MFPAD_totordered_S_mod, theory=False, a=2) #equal to the original!
    MFPAD_totordered_R_cos=fugi.sorting_array(MFPAD_totordered_R, theory=False, a=1)
    MFPAD_totordered_R_phi=fugi.sorting_array(MFPAD_totordered_R, theory=False, a=2) #equal to the original!

    #normalizing
    MFPAD_RCLCR_sum_norm=fugi.normalise_with_err(MFPAD_RCLCR_sum,normtype=2)
    MFPAD_SCLCR_sum_norm=fugi.normalise_with_err(MFPAD_SCLCR_sum,normtype=2)
    MFPAD_totordered_S_phi_norm3=fugi.normalise_with_err(MFPAD_totordered_S_phi,normtype=3)
    MFPAD_totordered_S_phi_norm=fugi.normalise_with_err(MFPAD_totordered_S_phi,normtype=2)
    MFPAD_totordered_S_mod_phi_norm=fugi.normalise_with_err(MFPAD_totordered_S_mod_phi,normtype=2)
    MFPAD_totordered_R_phi_norm=fugi.normalise_with_err(MFPAD_totordered_R_phi,normtype=2)

    temp=[]
    # for el in temp_totordered_modMFPAD.reshape(72,nphi,ncos):
    for el in temp_totordered_R.reshape(72,nphi,ncos):
        temp.append(fugi.projection(el, alongaxis=0))
    ctheta_RS_phi_norm=np.array(temp).reshape(72,ncos)
    
    for j in range(12):
        for i in range(6):
            if j>5:
                temp_PECD_S[j,i]=np.divide(np.subtract(gaussian_filter(MFPAD_SCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(MFPAD_SCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[j-6,5-i]),np.add(gaussian_filter(MFPAD_SCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(MFPAD_SCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[j-6,5-i]))
                temp_PECD_SR[j,i]=np.divide(np.subtract(MFPAD_totordered_S_phi_norm.reshape(12,6,-1)[j,i],MFPAD_totordered_S_phi_norm.reshape(12,6,-1)[j-6,5-i]),np.add(MFPAD_totordered_S_phi_norm.reshape(12,6,-1)[j,i],MFPAD_totordered_S_phi_norm.reshape(12,6,-1)[j-6,5-i]))
                temp_PECD_SR_mod[j,i]=np.divide(np.subtract(MFPAD_totordered_S_mod_phi_norm.reshape(12,6,-1)[j,i],MFPAD_totordered_S_mod_phi_norm.reshape(12,6,-1)[j-6,5-i]),np.add(MFPAD_totordered_S_mod_phi_norm.reshape(12,6,-1)[j,i],MFPAD_totordered_S_mod_phi_norm.reshape(12,6,-1)[j-6,5-i]))
                temp_PECD_RS[j,i]=np.divide(np.subtract(MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[j,i],MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[j-6,5-i]),np.add(MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[j,i],MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[j-6,5-i]))
                # temp_PECD_RS[j,i]=np.divide(np.subtract(MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[j-6,5-i],MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[j,i]),np.add(MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[j-6,5-i],MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[j,i]))
                temp_PECD_SR_sm[j,i]=np.divide(np.subtract(gaussian_filter(MFPAD_totordered_S_mod_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(MFPAD_totordered_S_mod_phi_norm,sigma=gaussian).reshape(12,6,-1)[j-6,5-i]),np.add(gaussian_filter(MFPAD_totordered_S_mod_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(MFPAD_totordered_S_mod_phi_norm,sigma=gaussian).reshape(12,6,-1)[j-6,5-i]))
                temp_cPECD_RS[j,i]=np.divide(np.subtract(ctheta_RS_phi_norm.reshape(12,6,-1)[j,i],ctheta_RS_phi_norm.reshape(12,6,-1)[j-6,5-i]),np.add(ctheta_RS_phi_norm.reshape(12,6,-1)[j,i],ctheta_RS_phi_norm.reshape(12,6,-1)[j-6,5-i]))
                temp_cPECD_RS_sm[j,i]=np.divide(np.subtract(gaussian_filter(ctheta_RS_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(ctheta_RS_phi_norm,sigma=gaussian).reshape(12,6,-1)[j-6,5-i]),np.add(gaussian_filter(ctheta_RS_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(ctheta_RS_phi_norm,sigma=gaussian).reshape(12,6,-1)[j-6,5-i]))
                temp_cPECD_err_SR[j,i]=fugi.error_calc(ctheta_RS_phi_norm.reshape(12,6,-1)[j,i],ctheta_RS_phi_norm.reshape(12,6,-1)[j-6,5-i])
            else:
                temp_PECD_S[j,i]=np.divide(np.subtract(gaussian_filter(MFPAD_SCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(MFPAD_SCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[6+j,5-i]),np.add(gaussian_filter(MFPAD_SCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(MFPAD_SCLCR_phi_norm,sigma=gaussian).reshape(12,6,-1)[6+j,5-i]))
                temp_PECD_SR[j,i]=np.divide(np.subtract(MFPAD_totordered_S_phi_norm.reshape(12,6,-1)[j,i],MFPAD_totordered_S_phi_norm.reshape(12,6,-1)[6+j,5-i]),np.add(MFPAD_totordered_S_phi_norm.reshape(12,6,-1)[j,i],MFPAD_totordered_S_phi_norm.reshape(12,6,-1)[6+j,5-i]))
                temp_PECD_SR_mod[j,i]=np.divide(np.subtract(MFPAD_totordered_S_mod_phi_norm.reshape(12,6,-1)[j,i],MFPAD_totordered_S_mod_phi_norm.reshape(12,6,-1)[6+j,5-i]),np.add(MFPAD_totordered_S_mod_phi_norm.reshape(12,6,-1)[j,i],MFPAD_totordered_S_mod_phi_norm.reshape(12,6,-1)[6+j,5-i]))
                temp_PECD_RS[j,i]=np.divide(np.subtract(MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[j,i],MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[6+j,5-i]),np.add(MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[j,i],MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[6+j,5-i]))
                # temp_PECD_RS[j,i]=np.divide(np.subtract(MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[6+j,5-i],MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[j,i]),np.add(MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[6+j,5-i],MFPAD_totordered_R_phi_norm.reshape(12,6,-1)[j,i]))
                temp_PECD_SR_sm[j,i]=np.divide(np.subtract(gaussian_filter(MFPAD_totordered_S_mod_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(MFPAD_totordered_S_mod_phi_norm,sigma=gaussian).reshape(12,6,-1)[6+j,5-i]),np.add(gaussian_filter(MFPAD_totordered_S_mod_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(MFPAD_totordered_S_mod_phi_norm,sigma=gaussian).reshape(12,6,-1)[6+j,5-i]))
                temp_cPECD_RS[j,i]=np.divide(np.subtract(ctheta_RS_phi_norm.reshape(12,6,-1)[j,i],ctheta_RS_phi_norm.reshape(12,6,-1)[6+j,5-i]),np.add(ctheta_RS_phi_norm.reshape(12,6,-1)[j,i],ctheta_RS_phi_norm.reshape(12,6,-1)[6+j,5-i]))
                temp_cPECD_RS_sm[j,i]=np.divide(np.subtract(gaussian_filter(ctheta_RS_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(ctheta_RS_phi_norm,sigma=gaussian).reshape(12,6,-1)[6+j,5-i]),np.add(gaussian_filter(ctheta_RS_phi_norm,sigma=gaussian).reshape(12,6,-1)[j,i],gaussian_filter(ctheta_RS_phi_norm,sigma=gaussian).reshape(12,6,-1)[6+j,5-i]))
                temp_cPECD_err_SR[j,i]=fugi.error_calc(ctheta_RS_phi_norm.reshape(12,6,-1)[j,i],ctheta_RS_phi_norm.reshape(12,6,-1)[6+j,5-i])

    PECD_SCLCR=np.array(temp_PECD_S).reshape(72,nphi,ncos)
    PECD_SR=np.array(temp_PECD_SR).reshape(72,nphi,ncos)
    PECD_SR_mod=np.array(temp_PECD_SR_mod).reshape(72,nphi,ncos)
    PECD_SR_sm=np.array(temp_PECD_SR_sm).reshape(72,nphi,ncos)
    PECD_RS=np.array(temp_PECD_RS).reshape(72,nphi,ncos) #to be comapred with MOx

    cPECD_norm_phi=np.array(temp_cPECD_RS).reshape(72,-1)
    cPECD_norm_phi_sm=np.array(temp_cPECD_RS_sm).reshape(72,-1)
    errPECD_phi=np.array(temp_cPECD_err_SR/100).reshape(72,-1) #works without dividing 100

    mPECD = np.divide(np.subtract(MFPAD_RCLCR_sum_norm,MFPAD_SCLCR_sum_norm),np.add(MFPAD_RCLCR_sum_norm,MFPAD_SCLCR_sum_norm))
    mPECD_smooth = np.divide(np.subtract(gaussian_filter(MFPAD_RCLCR_sum_norm,sigma=gaussian),gaussian_filter(MFPAD_SCLCR_sum_norm,sigma=gaussian)),np.add(gaussian_filter(MFPAD_RCLCR_sum_norm,sigma=gaussian),gaussian_filter(MFPAD_SCLCR_sum_norm,sigma=gaussian)))

    #! this is an R-enant - S-enant and exist just for 11.7 eV
    mPECD_hires = np.divide(np.subtract(MFPAD_tot_sum_norm,MFPAD_tot_sum_flip_norm),np.add(MFPAD_tot_sum_norm,MFPAD_tot_sum_flip_norm))
    mPECD_hires_smooth = np.divide(np.subtract(gaussian_filter(MFPAD_tot_sum_norm,sigma=gaussian),gaussian_filter(MFPAD_tot_sum_flip_norm,sigma=gaussian)),np.add(gaussian_filter(MFPAD_tot_sum_norm,sigma=gaussian),gaussian_filter(MFPAD_tot_sum_flip_norm,sigma=gaussian)))

    #test wothout sorting j,i
    # cPECD_phi_test = np.nan_to_num(np.divide(np.subtract(ctheta_R_phi_norm,ctheta_S_phi_norm),np.add(ctheta_R_phi_norm,ctheta_S_phi_norm))) #TEST
    # errPECD_phi_LF = np.nan_to_num(np.sqrt(1/(ctheta_R_phi_norm+ctheta_S_phi_norm)))

else:
    MFPAD_totordered_R=MFPAD_RCLCR
    MFPAD_totordered_R_cos=MFPAD_RCLCR_cos
    MFPAD_totordered_R_phi=MFPAD_RCLCR_phi

    MFPAD_totordered_R=np.array(MFPAD_RCLCR_flip).reshape(72,nphi,ncos)

    #PECD check it is equal to contrast; has to be normalized to avoid the contribution of unballanced counts
    MFPAD_totordered_R_sum_norm=fugi.normalise_with_err(np.sum(MFPAD_totordered_R,axis=0),normtype=2)
    MFPAD_totordered_R_sum_flip_norm=fugi.normalise_with_err(np.sum(MFPAD_totordered_R,axis=0),normtype=2)

    mPECD=np.divide(np.subtract(MFPAD_totordered_R_sum_norm,MFPAD_totordered_R_sum_flip_norm),np.add(MFPAD_totordered_R_sum_norm,MFPAD_totordered_R_sum_flip_norm))
    mPECD_smooth=np.divide(np.subtract(gaussian_filter(MFPAD_totordered_R_sum_norm,sigma=gaussian),gaussian_filter(MFPAD_totordered_R_sum_flip_norm,sigma=gaussian)),np.add(gaussian_filter(MFPAD_totordered_R_sum_norm,sigma=gaussian),gaussian_filter(MFPAD_totordered_R_sum_flip_norm,sigma=gaussian)))

In [ ]:
#attempt to replicate the results of PECD oriented in space (not fixed): integration of PECD to resamble the diatomic
testRtot=np.sum((np.sum(MFPAD_RCLCR_phi_norm.reshape(12,6,-1),axis=0)).reshape(6,nphi,ncos),axis=1)
PECD_test_RCLCR=np.sum((np.sum(PECD_RCLCR.reshape(12,6,-1),axis=0)).reshape(6,nphi,ncos),axis=1)

if fileSCL and fileSCR is not None:
    testStot=np.sum((np.sum(MFPAD_SCLCR_phi_norm.reshape(12,6,-1),axis=0)).reshape(6,nphi,ncos),axis=1)
    PECD_test_SCLCR=np.sum((np.sum(PECD_SCLCR.reshape(12,6,-1),axis=0)).reshape(6,nphi,ncos),axis=1)
    PECD_test = np.nan_to_num(np.divide(np.subtract(testStot,testRtot),np.add(testStot,testRtot))) #TEST

if fileSCL and fileSCR is not None:
    plt.pcolormesh(PECD_test_SCLCR.T)
else:
    plt.pcolormesh(PECD_test_RCLCR.T)

plt.imshow(np.sum(temp_RCLCR_flip.reshape(72,-1),axis=0).reshape(36,18).T)

plt.imshow(np.sum(temp_SCLCR.reshape(72,-1),axis=0).reshape(36,18).T)

plt.imshow(np.flip(MFPAD_tot_sum,axis=1).T)

In [ ]:
# fugi = reload(fugi)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,4))
if fileSCL and fileSCR is not None:
    ax1.contourf(phiM,cosM,MFPAD_SCLCR[7].T)
    #! S-enantiomer
    ax3.contourf(phiM,cosM,MFPAD_tot_sum_flip_norm.T)

ax2.contourf(phiM,cosM,MFPAD_RCLCR[13].T)
# cs,ax2=fugi.plot_interpolation(phiM,cosM,MFPAD_RCR[71].T, ax=ax2, kind="linear", xstep=30, ystep=0.1, n=8)
# MFPAD_SCL_flip_cos[45]==np.flip(np.flip(MFPAD_SCL_cos[45],axis=1),axis=0) #True

#test samrt sum
counts=0
for j in range(12):
    for i in range(6):
        if j>5:  
            print("b=",i,j,"a1=",5-i,j-6)
            i+=1
        else:
            print("b=",i,j,"a2=",5-i,j+6)
            i+=1
        counts+=1
print(counts)
#check the reshape(6,12,-1)
cosphi_adj_cos.reshape(6,12,-1)[:13]


In [ ]:
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    cs,ax=fugi.plot_interpolation(phiM,cosM,(PECD_RCLCR[i]*100).T, ax=ax, cmap=cmap_temp, limits=[-40.,40.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
    ax.tick_params(labelsize=12)

    if i == 4:
        ax.spines[:].set_color('blue')
        ax.spines[:].set_linewidth(5)
    elif i == 37:
        ax.spines[:].set_color('red')
        ax.spines[:].set_linewidth(5)
    
cbar = fig.colorbar(cs, ax=axes.ravel().tolist(), anchor=(1.5,1), pad=-2.5)
cbar.ax.set_ylabel('PECD %')

fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/72PECD_"+CH+"_"+ID+"_"+en+"_RCLCR_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    # plt.show()
    plt.close()
else:
    plt.show()

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    cs,ax=fugi.plot_interpolation(phiM,cosM,(PECD_RCLCR_smooth[i]*100).T, ax=ax, cmap=cmap_temp, limits=[-40.,40.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
    ax.tick_params(labelsize=12)

    if i == 4:
        ax.spines[:].set_color('blue')
        ax.spines[:].set_linewidth(5)
    elif i == 37:
        ax.spines[:].set_color('red')
        ax.spines[:].set_linewidth(5)
    
cbar = fig.colorbar(cs, ax=axes.ravel().tolist(), anchor=(1.5,1), pad=-2.5)
cbar.ax.set_ylabel('PECD %')

fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/72PECD_"+CH+"_"+ID+"_"+en+"_RCLCR_smooth_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    # plt.show()
    plt.close()
else:
    plt.show()

In [ ]:
fig,ax = plt.subplots(2,1, figsize=(10, 18), sharex=True)
aspect=(phiM.max()/cosM.max())


cs,ax[0]=fugi.plot_interpolation(phiM,cosM,(PECD_RCLCR[0]*100).T, ax=ax[0], cmap=cmap_temp, limits=[-40.,40.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
ax[0].set_ylabel('cos\u03D1 [adm]')
# ax[0].set_xlabel('\u03C6 [DEG]')
ax[0].set_aspect(aspect)

cs,ax[1]=fugi.plot_interpolation(phiM,cosM,(PECD_RCLCR[4]*100).T, ax=ax[1], cmap=cmap_temp, limits=[-40.,40.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
ax[1].set_ylabel('cos\u03D1 [adm]')
ax[1].set_xlabel('\u03C6 [DEG]')
ax[1].set_aspect(aspect)

cbar = fig.colorbar(cs, ax=ax.ravel().tolist(), location="top", orientation="horizontal", shrink=0.6)
cbar.ax.set_xlabel('PECD %', ma="right")

fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/0-4PECD_"+CH+"_"+ID+"_"+en+"_RCLCR_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    plt.close()
    # plt.show()
else:
    plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        cs,ax=fugi.plot_interpolation(phiM,cosM,(PECD_SCLCR[i]*100).T, ax=ax, cmap=cmap_temp, limits=[-40.,40.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
        ax.tick_params(labelsize=12)

        if i == 4:
            ax.spines[:].set_color('blue')
            ax.spines[:].set_linewidth(5)
        elif i == 37:
            ax.spines[:].set_color('red')
            ax.spines[:].set_linewidth(5)
    
    cbar = fig.colorbar(cs, ax=axes.ravel().tolist(), anchor=(1.5,1), pad=-2.5)
    cbar.ax.set_ylabel('PECD %')

    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/72PECD_"+CH+"_"+ID+"_"+en+"_SCLCR_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')
    if noprint:
        # plt.show()
        plt.close()
    else:
        plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        # ax.contourf(phiM,cosM,fugi.smoothgauss(PECD_SR_mod[i],0.3,0.6).T, cmap=cmap_temp)
        cs,ax=fugi.plot_interpolation(phiM,cosM,(PECD_SR_mod[i]*100).T, ax=ax, cmap=cmap_temp, limits=[-40.,40.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
        ax.tick_params(labelsize=12)
        
        if i == 4:
            ax.spines[:].set_color('blue')
            ax.spines[:].set_linewidth(5)
        elif i == 37:
            ax.spines[:].set_color('red')
            ax.spines[:].set_linewidth(5)
    
    cbar = fig.colorbar(cs, ax=axes.ravel().tolist(), ticks=mticker.MultipleLocator(10), anchor=(1.5,1), pad=-2.5)
    cbar.ax.set_ylabel('PECD %')

    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/72PECD_"+CH+"_"+ID+"_"+en+"_SR_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')
    if noprint:
        plt.show()
        # plt.close()
    else:
        plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    #this can be all added togheter to create the total sum
    # fig,ax = plt.subplots(2,1, figsize=(8, 12), sharex='col', tight_layout=True)
    fig,ax = plt.subplots(2,1, figsize=(10, 18), sharex=True)
    aspect=(phiM.max()/cosM.max())

    cs,ax[0]=fugi.plot_interpolation(phiM,cosM,(PECD_SR_mod[0]*100).T, ax=ax[0], cmap=cmap_temp, limits=[-40.,40.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
    ax[0].set_ylabel('cos\u03D1 [adm]')
    # ax[0].set_xlabel('\u03C6 [DEG]')
    ax[0].set_aspect(aspect)

    cs,ax[1]=fugi.plot_interpolation(phiM,cosM,(PECD_SR_mod[4]*100).T, ax=ax[1], cmap=cmap_temp, limits=[-40.,40.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
    ax[1].set_ylabel('cos\u03D1 [adm]')
    ax[1].set_xlabel('\u03C6 [DEG]')
    ax[1].set_aspect(aspect)

    cbar = fig.colorbar(cs, ax=ax.ravel().tolist(), location="top", orientation="horizontal", shrink=0.6)
    cbar.ax.set_xlabel('PECD %', ma="right")
    
    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/0-4PECD_"+CH+"_"+ID+"_"+en+"_SR_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')
    
    if noprint:
        # plt.close()
        plt.show()
    else:
        plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        # ax.contourf(phiM,cosM,fugi.smoothgauss(PECD_SR_sm[i],0.3,0.6).T, cmap=cmap_temp)
        cs,ax=fugi.plot_interpolation(phiM,cosM,(PECD_SR_sm[i]*100).T, ax=ax, cmap=cmap_temp, limits=[-35.,35.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
        ax.tick_params(labelsize=12)

        if i == 4:
            ax.spines[:].set_color('blue')
            ax.spines[:].set_linewidth(5)
        elif i == 37:
            ax.spines[:].set_color('red')
            ax.spines[:].set_linewidth(5)
    
    cbar = fig.colorbar(cs, ax=axes.ravel().tolist(), ticks=mticker.MultipleLocator(10), anchor=(1.5,1), pad=-2.5)
    cbar.ax.set_ylabel('PECD %')

    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/72PECD_"+CH+"_"+ID+"_"+en+"_SR_phi_gauss_exp.png", dpi=300, transparent=False, bbox_inches='tight')
    if noprint:
        # plt.show()
        plt.close()
    else:
        plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    # syntax: fugi = reload(fugi)
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        cs,ax=fugi.plot_interpolation(phiM,cosM,(PECD_RS[i]*100).T, ax=ax, cmap=cmap_temp, limits=[-40.,40.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
        ax.tick_params(labelsize=12)

        if i == 4:
            ax.spines[:].set_color('blue')
            ax.spines[:].set_linewidth(5)
        elif i == 37:
            ax.spines[:].set_color('red')
            ax.spines[:].set_linewidth(5)
    
    cbar = fig.colorbar(cs, ax=axes.ravel().tolist(), anchor=(1.5,1), pad=-2.5)
    cbar.ax.set_ylabel('PECD %')

    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/72PECD_"+CH+"_"+ID+"_"+en+"_RScompariosnMOX_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')
    if noprint:
        # plt.show()
        plt.close()
    else:
        plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    #SUM of S enantiomers two different helicities
    # fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        #old adding up
        # ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_SCLCR_phi[i],0.3,0.6).T, levels=5)
        cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_SCLCR_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
        ax.tick_params(labelsize=12)
        # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

    fugi.overlaycbar(fig,cs,axes,MFPAD=True)
    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/72MFPAD_"+CH+"_"+ID+"_"+en+"_SCLCR_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')
    if noprint:
        plt.close()
    else:
        plt.show()

In [ ]:
#SUM of R enantiomers two different helicities
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    #old adding up
    # ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_RCLCR_phi[i],0.3,0.6).T)
    cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_RCLCR_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)

    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

fugi.overlaycbar(fig,cs,axes,MFPAD=True)
fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/72MFPAD_"+CH+"_"+ID+"_"+en+"_RCLCR_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    plt.close()
else:
    plt.show()

In [ ]:
#TOTAL sum of enantiomers and helicities smoothed
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    # ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_totordered_R_phi[i],0.3,0.6).T)
    cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_totordered_R_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

fugi.overlaycbar(fig,cs,axes,MFPAD=True)
fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/72MFPAD_"+CH+"_"+ID+"_"+en+"_total_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    plt.close()
    # plt.show()
else:
    plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    # fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        # ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_totordered_S[i],0.3,0.6).T)
        cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_totordered_S[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
        ax.tick_params(labelsize=12)
        # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

    fugi.overlaycbar(fig,cs,axes,MFPAD=True)
    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/72MFPAD_"+CH+"_"+ID+"_"+en+"_total_S_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')

    if noprint:
        plt.close()
        # plt.show()
    else:
        plt.show()

In [ ]:
#this can be all added togheter to create the total sum
# fig,ax = plt.subplots(2,1, figsize=(8, 12), sharex='col', tight_layout=True)
fig,ax = plt.subplots(2,1, figsize=(10, 18), sharex=True)
aspect=(phiM.max()/cosM.max())

cs,ax[0]=fugi.plot_interpolation(phiM,cosM,MFPAD_totordered_R_phi[37].T, ax=ax[0], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
contrast=cs.get_array().max()/cs.get_array().min()
ax[0].set_ylabel('cos\u03D1 [adm]')
# ax[0].set_xlabel('\u03C6 [DEG]')
ax[0].set_aspect(aspect)

cs,ax[1]=fugi.plot_interpolation(phiM,cosM,MFPAD_totordered_R_phi[4].T, ax=ax[1], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
ax[1].set_ylabel('cos\u03D1 [adm]')
ax[1].set_xlabel('\u03C6 [DEG]')
ax[1].set_aspect(aspect)

cbar = fig.colorbar(cs, ax=ax.ravel().tolist(), location="top", orientation="horizontal", shrink=0.6)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
# cbar.ax.set_ylabel('normalised counts', rotation="horizontal", rotation_mode="anchor")

fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/37-4MFPAD_"+CH+"_"+ID+"_"+en+"_total_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    # plt.close()
    plt.show()
else:
    plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    #this can be all added togheter to create the total sum
    # fig,ax = plt.subplots(2,1, figsize=(8, 12), sharex='col', tight_layout=True)
    fig,ax = plt.subplots(2,1, figsize=(10, 18), sharex=True)
    aspect=(phiM.max()/cosM.max())

    cs,ax[0]=fugi.plot_interpolation(phiM,cosM,MFPAD_totordered_S_phi[37].T, ax=ax[0], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
    contrast=cs.get_array().max()/cs.get_array().min()
    ax[0].set_ylabel('cos\u03D1 [adm]')
    # ax[0].set_xlabel('\u03C6 [DEG]')
    ax[0].set_aspect(aspect)

    cs,ax[1]=fugi.plot_interpolation(phiM,cosM,MFPAD_totordered_S_phi[4].T, ax=ax[1], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
    ax[1].set_ylabel('cos\u03D1 [adm]')
    ax[1].set_xlabel('\u03C6 [DEG]')
    ax[1].set_aspect(aspect)

    cbar = fig.colorbar(cs, ax=ax.ravel().tolist(), location="top", orientation="horizontal", shrink=0.6)
    cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
    cbar.set_ticklabels([1,f"{contrast:.2f}"])
    # cbar.ax.set_ylabel('normalised counts', rotation="horizontal", rotation_mode="anchor")

    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/37-4MFPAD_"+CH+"_"+ID+"_"+en+"_total_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')

    if noprint:
        # plt.close()
        plt.show()
    else:
        plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    #TOTAL sum of enantiomers and helicities smoothed
    # fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
    
    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        # ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_totordered_S_mod_phi[i],0.3,0.6).T)
        cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_totordered_S_mod_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
        ax.tick_params(labelsize=12)
        # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
    
    fugi.overlaycbar(fig,cs,axes,MFPAD=True)
    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/72MFPAD_"+CH+"_"+ID+"_"+en+"_total_S_MOD_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')
    
    if noprint:
        plt.close()
        # plt.show()
    else:
        plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    #this can be all added togheter to create the total sum
    # fig,ax = plt.subplots(2,1, figsize=(8, 12), sharex='col', tight_layout=True)
    fig,ax = plt.subplots(2,1, figsize=(10, 18), sharex=True)
    aspect=(phiM.max()/cosM.max())
    
    cs,ax[0]=fugi.plot_interpolation(phiM,cosM,MFPAD_totordered_S_mod_phi[37].T, ax=ax[0], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
    contrast=cs.get_array().max()/cs.get_array().min()
    # cs,ax[0]=fugi.plot_interpolation(phiM,cosM,MFPAD_SCL_phi[4].T, ax=ax[0], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
    ax[0].set_ylabel('cos\u03D1 [adm]')
    # ax[0].set_xlabel('\u03C6 [DEG]')
    ax[0].set_aspect(aspect)
    
    cs,ax[1]=fugi.plot_interpolation(phiM,cosM,MFPAD_totordered_S_mod_phi[4].T, ax=ax[1], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
    # cs,ax[1]=fugi.plot_interpolation(phiM,cosM,MFPAD_SCR_phi[4].T, ax=ax[1], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)

    ax[1].set_ylabel('cos\u03D1 [adm]')
    ax[1].set_xlabel('\u03C6 [DEG]')
    ax[1].set_aspect(aspect)
    
    cbar = fig.colorbar(cs, ax=ax.ravel().tolist(), location="top", orientation="horizontal", shrink=0.6)
    cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
    cbar.set_ticklabels([1,f"{contrast:.2f}"])
    # cbar.ax.set_ylabel('normalised counts', rotation="horizontal", rotation_mode="anchor")
    
    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/37-4MFPAD_"+CH+"_"+ID+"_"+en+"_total_S_mod_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')
    
    if noprint:
        # plt.close()
        plt.show()
    else:
        plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    MFPAD_SCL_sum_n2=fugi.normalise_with_err(MFPAD_SCL_sum,normtype=2)
    fig,ax= plt.subplots(1, figsize=(12, 10))
    # cs=ax.contourf(phiM,cosM,MFPAD_SCL_sum_n2.T, cmap=cmap_temp, levels=20)
    cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_SCL_sum_n2.T, ax=ax, cmap=cmap_temp, xstep=1, ystep=0.01, n=20, limits=True)
    contrast=cs.get_array().max()/cs.get_array().min()

    ax.set_title("SCL tot MFPAD "+ID+" "+CH+"_"+en)

    aspect=(phiM.max()/cosM.max())
    ax.set_xlabel('\u03C6 [DEG]')
    ax.set_ylabel('cos\u03D1 [adm]')
    cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
    cbar.set_ticks([MFPAD_SCL_sum_n2.min(),MFPAD_SCL_sum_n2.max()])
    cbar.set_ticklabels([1,f"{contrast:.2f}"])
    cbar.ax.set_ylabel('normalised counts')
    ax.set_aspect(aspect)

    # fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/SCL_MFPAD_single_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

#a try with polotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

endrange=[]

for MFPAD in MFPAD_RCR_cos:
    endrange.append(np.amax(fugi.smoothgauss(MFPAD,0.3,0.6)))
endrange=np.amax(endrange)


fig = go.Figure()
fig = make_subplots(rows=6, cols=12, vertical_spacing=0.015)

for i in range(12):
    for j in range(6):
        fig.add_trace(go.Contour(x=phiM,y=cosM,z=fugi.smoothgauss(MFPAD_RCR_cos[j+i*6],0.3,0.6).T, line_smoothing=0.7, legendgroup="no", colorscale="Viridis", showscale=False, showlegend=False, contours=dict(start=0., end=endrange, size=endrange/10)), row=j+1, col=i+1)
    
fig.update_layout(
    title=dict(
        text="MFPAD smoothed cos sorted" ,
        #y=0.9,
        x=0.5,
        xanchor="center",
        yanchor="top"),
    #coloraxis=dict(colorscale=Seismic_r),
    #showlegend=False,
    autosize=True,
    width=1000,
    #height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()

In [ ]:
if fileSCL and fileSCR is not None:
    #this can be all added togheter to create the total sum
    fig,ax = plt.subplots(2,2, figsize=(18, 15), sharex=True, sharey=True, tight_layout=True)
    st = fig.suptitle("MFPAD RS "+ID+" "+CH+" "+en, fontsize="x-large")
    aspect=(phiM.max()/cosM.max())

    cs1=ax[0][0].contourf(phiM,cosM,MFPAD_RCR_sum.T,cmap=cmap_temp)
    ax[0][0].set_title("RCR")
    ax[0][0].set_ylabel('cos\u03D1 [adm]')
    # fig.colorbar(cs1, extend='both', shrink=0.9, ax=ax[0][0])
    ax[0][0].set_aspect(aspect)

    cs2=ax[0][1].contourf(phiM,cosM,MFPAD_SCR_flip_sum.T,cmap=cmap_temp)
    ax[0][1].set_title("SCR_flip")
    # fig.colorbar(cs2,extend='both', shrink=0.9, ax=ax[0][1])
    ax[0][1].set_aspect(aspect)

    cs3=ax[1][0].contourf(phiM,cosM,MFPAD_RCL_sum.T,cmap=cmap_temp)
    ax[1][0].set_title("RCL")
    ax[1][0].set_xlabel('phi [DEG]')
    ax[1][0].set_ylabel('cos\u03D1 [adm]')
    # fig.colorbar(cs3,extend='both', shrink=0.9, ax=ax[1][0])
    ax[1][0].set_aspect(aspect)

    cs4=ax[1][1].contourf(phiM,cosM,MFPAD_SCL_flip_sum.T,cmap=cmap_temp)
    ax[1][1].set_title("SCL_flip")
    ax[1][1].set_xlabel('phi [DEG]')
    # fig.colorbar(cs4,extend='both', shrink=0.9, ax=ax[1][1])
    ax[1][1].set_aspect(aspect)

    st.set_y(0.95)
    # fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/total_MFPAD_sum_RS_test_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
#this can be all added togheter to create the total sum
fig,ax = plt.subplots(1,2, figsize=(25, 10))
# st = fig.suptitle("MFPAD "+CH+"_"+en, fontsize="x-large", y=0.9)
aspect=(phiM.max()/cosM.max())

# cs1=ax[0].contourf(phiM,cosM,MFPAD_RCR_sum.T,cmap=cmap_temp)
cs0,ax[0]=fugi.plot_interpolation(phiM,cosM,MFPAD_R_sum.T, ax=ax[0], xstep=1, ystep=0.01, n=20, gaussian=7., limits=True)
contrast=cs0.get_array().max()/cs0.get_array().min()

ax[0].set_title("MFPAD R CLCR "+ID+" "+CH+" "+en)
ax[0].set_ylabel('cos\u03D1 [adm]')
ax[0].set_xlabel('\u03C6 [DEG]')
ax[0].set_aspect(aspect)
cbar0=fig.colorbar(cs0, shrink=0.9, ax=ax[0])
cbar0.set_ticks([MFPAD_R_sum.min(),MFPAD_R_sum.max()])
cbar0.set_ticklabels([1,f"{contrast:.2f}"])

if fileSCL and fileSCR is not None:
    # cs1=ax[1].contourf(phiM,cosM,MFPAD_RCL_sum.T,cmap=cmap_temp)
    cs1,ax[1]=fugi.plot_interpolation(phiM,cosM,np.flip(MFPAD_S_sum,axis=1).T, ax=ax[1], xstep=1, ystep=0.01, n=20, gaussian=7., limits=True)
    contrast=cs1.get_array().max()/cs1.get_array().min()

    ax[1].set_title("MFPAD S CLCR "+ID+" "+CH+" "+en)
    ax[1].set_ylabel('cos\u03D1 [adm]')
    ax[1].set_xlabel('\u03C6 [DEG]')
    ax[1].set_aspect(aspect)
    cbar1 = fig.colorbar(cs1, ax=ax[1],shrink=0.9)
    cbar1.set_ticks([MFPAD_S_sum.min(),MFPAD_S_sum.max()])
    cbar1.set_ticklabels([1,f"{contrast:.2f}"])
    cbar1.ax.set_ylabel('normalised counts')

fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/total_MFPAD_RvS_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
# cs=ax.contourf(phiM,cosM,MFPAD_tot_sum.T, cmap=cmap_temp, levels=15)
cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_tot_sum.T, ax=ax, cmap=cmap_temp, xstep=1, ystep=0.01, n=20)
contrast=cs.get_array().max()/cs.get_array().min()
ax.set_title("MFPAD R+S "+ID+" "+CH+" "+en)

aspect=(phiM.max()/cosM.max())
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/total_MFPAD_R-S-CLCR_cmap_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
%%capture
fig,ax= plt.subplots(1, figsize=(12, 10))
# cs=ax.contourf(phiM,cosM,MFPAD_tot_sum.T, cmap=cmap_temp, levels=15)
cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_tot_sum.T, ax=ax, xstep=1, ystep=0.01, n=20)
contrast=cs.get_array().max()/cs.get_array().min()
ax.set_title("MFPAD R+S "+ID+" "+CH+" "+en)

aspect=(phiM.max()/cosM.max())
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/total_MFPAD_R-S-CLCR_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
# cs=ax.contourf(phiM,cosM,MFPAD_tot_sum.T, cmap=cmap_temp, levels=15)
cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_tot_sum.T, ax=ax, cmap=cmap_temp, xstep=1, ystep=0.01, n=20, gaussian=7.)
contrast=cs.get_array().max()/cs.get_array().min()
ax.set_title("MFPAD R+S "+ID+" "+CH+" "+en)

aspect=(phiM.max()/cosM.max())
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/total_MFPAD_R-S-CLCR_gauss_cmap_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
%%capture
fig,ax= plt.subplots(1, figsize=(12, 10))
# cs=ax.contourf(phiM,cosM,MFPAD_tot_sum.T, cmap=cmap_temp, levels=15)
cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_tot_sum.T, ax=ax, xstep=1, ystep=0.01, n=20, gaussian=7.)
contrast=cs.get_array().max()/cs.get_array().min()
ax.set_title("MFPAD R+S "+ID+" "+CH+" "+en)

aspect=(phiM.max()/cosM.max())
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/total_MFPAD_R-S-CLCR_gauss_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
%%capture
fig,ax= plt.subplots(1, figsize=(12, 10))
# cs=ax.contourf(phiM,cosM,MFPAD_tot_sum.T, cmap=cmap_temp, levels=15)
cs,ax=fugi.plot_interpolation(phiM,cosM,fugi.normalise_with_err(MFPAD_tot_sum).T, ax=ax, xstep=1, ystep=0.01, n=20, gaussian=7.)
ax.set_title("MFPAD R+S "+ID+" "+CH+" "+en)

aspect=(phiM.max()/cosM.max())
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.ax.set_ylabel('absolute counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/total_MFPAD_R-S-CLCR_rawcomparison_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
%%capture
fig,ax= plt.subplots(1, figsize=(12, 10))
# cs=ax.contourf(phiM,cosM,MFPAD_tot_sum.T, cmap=cmap_temp, levels=15)
cs=ax.pcolormesh(phiM,cosM,fugi.normalise_with_err(MFPAD_tot_sum).T, shading="flat")
ax.set_title("raw MFPAD R+S "+ID+" "+CH+" "+en)

aspect=(phiM.max()/cosM.max())
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
cbar.ax.ticklabel_format(style='sci')
# cbar.ax.set_ylabel('absolute counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/total_MFPAD_R-S-CLCR_raw_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
if fileSCL and fileSCR is not None:
    #cos(theta) seems inverted rispect to contour
    fig = make_subplots(rows=1, cols=3,
                        subplot_titles=("MFPAD tot R", "MFPAD SCLCR", "MFPAD RCLCR"))

    # fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2,     end=0.2, size=0.04)), 1, 1
    fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_tot_sum.T, line_smoothing=0.85, coloraxis="coloraxis1"), 1, 1)
    fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_SCLCR_sum.T, line_smoothing=0.85, coloraxis="coloraxis2"), 1, 2)
    fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_RCLCR_sum.T, line_smoothing=0.85, coloraxis="coloraxis3"), 1, 3)

    fig.update_layout(
        coloraxis1=dict(colorscale="Viridis",
                        colorbar=dict(x=0.335, xanchor='right', thickness=10)),
        coloraxis2=dict(colorscale="Viridis",
                        colorbar=dict(x=0.648, xanchor='left', thickness=10)),
        coloraxis3=dict(colorscale="Viridis",
                        colorbar=dict(x=1.0, xanchor='left', thickness=10)),
        # showlegend=False,
        autosize=False,
        width=1550,
        height=450,
        margin=dict(l=5,r=5,b=25,t=25)
        )

    fig.show()
else:

    #cos(theta) seems inverted rispect to contour
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=("MFPAD tot R", "MFPAD RCLCR"))

    # fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2,     end=0.2, size=0.04)), 1, 1
    fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_tot_sum.T, line_smoothing=0.85, coloraxis="coloraxis1"), 1, 1)
    fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_RCLCR_sum.T, line_smoothing=0.85, coloraxis="coloraxis2"), 1, 2)

    fig.update_layout(
        coloraxis1=dict(colorscale="Viridis",
                        colorbar=dict(x=0.525, xanchor='right', thickness=10)),
        coloraxis2=dict(colorscale="Viridis",
                        colorbar=dict(x=1.05, xanchor='left', thickness=10)),
        # showlegend=False,
        autosize=False,
        width=950,
        height=450,
        margin=dict(l=5,r=5,b=25,t=25)
        )

    fig.show()

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
# cs=ax.contourf(phiM,cosM,mPECD.T*100, cmap=cmap_temp, levels=10)
cs,ax=fugi.plot_interpolation(phiM,cosM,mPECD_smooth.T*100, ax=ax, cmap=cmap_temp, xstep=1, ystep=0.01, gaussian=5)

ax.set_title("Contrast map "+ID+" "+CH+" "+en)

aspect=(phiM.max()/cosM.max())
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
# cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
cbar.ax.set_ylabel('% of asymmetry')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/contrastmap_R-S-CLCR_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
if fileSCL and fileSCR is not None:
    fig,ax= plt.subplots(1, figsize=(12, 10))
    # cs=ax.contourf(phiM,cosM,mPECD_hires_smooth.T*100, cmap=cmap_temp, levels=10)
    cs,ax=fugi.plot_interpolation(phiM,cosM,mPECD_hires_smooth.T*100, ax=ax, cmap=cmap_temp, xstep=1, ystep=0.01)
    ax.set_title("Contrast map "+ID+" "+CH+" "+en)

    aspect=(phiM.max()/cosM.max())
    ax.set_xlabel('\u03C6 [DEG]')
    ax.set_ylabel('cos\u03D1 [adm]')
    cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
    # cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
    cbar.ax.set_ylabel('% of asymmetry')
    ax.set_aspect(aspect)

    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/contrastmap_R-S-CLCR_highres_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
if fileSCL and fileSCR is not None:
    fig,ax= plt.subplots(1, figsize=(12, 10))
    # cs=ax.contourf(phiM,cosM,mPECD_hires.T*100, cmap=cmap_temp, levels=10)
    cs,ax=fugi.plot_interpolation(phiM,cosM,mPECD_hires.T*100, ax=ax, cmap=cmap_temp, xstep=1, ystep=0.01, gaussian=5)
    ax.set_title("Contrast map "+ID+" "+CH+" "+en)

    aspect=(phiM.max()/cosM.max())
    ax.set_xlabel('\u03C6 [DEG]')
    ax.set_ylabel('cos(\u03D1) [adm]')
    cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
    # cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
    cbar.ax.set_ylabel('% of asymmetry')
    ax.set_aspect(aspect)

    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/contrastmap_R-S-CLCR_highres_gauss_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
#iminuit with error: NORMALIZED DATA
from scipy.interpolate import make_interp_spline, BSpline
import functions_minuit as fumi

if fileSCL and fileSCR is not None:
    #integrated in the function fumi.plotminuit
    # limits=[(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
    # limits=[(-1, 1), (-1, 2), (None, None), (None, None), (None, None), (None, None)]
    limits=[(None, None), (None, None), (None, None), (None, None), (None, None), (None, None)]

    m=fumi.iminuit_err(cosM,cPECD_norm_phi[0],errPECD_phi[0]*100,limits)

    print(m)
        
    x_data=cosM;y_data=cPECD_norm_phi[0]
    
    x_data_new = np.linspace(x_data.min(), x_data.max(), 300)
    bspl = make_interp_spline(x_data, fumi.PECD6(x_data, *m.values), k=5)
    
    plt.plot(x_data_new, bspl(x_data_new))
    plt.scatter(cosM,cPECD_norm_phi[0],color='r')
    plt.errorbar(cosM,cPECD_norm_phi[0], yerr=errPECD_phi[0]*100, fmt="o")
    plt.xlabel('cos(theta)_photon')
    plt.ylabel('PECD')
    
    plt.show()
    for p in m.parameters:
        print("{} = {:2.3e} +/- {:2.3e}".format(p, m.values[p], m.errors[p]))

In [ ]:
if fileSCL and fileSCR is not None:
    #S phi ordered modified with error
    # param_matrix=np.zeros((6,72)) # alternative nparray initialization
    param_matrix_phi = [[0 for i in range(6)] for j in range(72)] #i col, j row
    
    # fig, axes = plt.subplots(6,12, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
    fig, axes = plt.subplots(6,12, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
    custom_xlim = (-1, 1)
    custom_ylim = (-0.2,0.2) #custom_ylim = (-1,1)
    
    # limits=[(-1, 1), (-1, 2), (None, None), (-1, 2), (None, None), (-1, 2)]
    limits=[(-1, 1), (-1, 2), (None, None), (None, None), (None, None), (None, None)]
    
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    # for i,ax in enumerate(axes.ravel()): #for cos
    
        x_data = cosM; y_data = cPECD_norm_phi[i];y_err = errPECD_phi[i]*100
        
        m=fumi.iminuit_err(x_data,y_data,y_err,limits)
        fumi.plotminuit(x_data,y_data,y_err,m,ax,custom_xlim,custom_ylim)
    
        for counter,el in enumerate(m.parameters): # len = 6
            param_matrix_phi[i][counter]=((m.values[el], m.errors[el]))
    
    param_matrix_phi=np.array(param_matrix_phi) #numpy tranformation!!!
    
    fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)
    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/b1matrix_R-S_phi_smooth_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')
        # for p in m.parameters:
        #     print("{} = {:2.3e} +/- {:2.3e}".format(p, m.values[p], m.errors[p]))

In [ ]:
if fileSCL and fileSCR is not None:
    # fugi = reload(fugi)
    fig,ax= plt.subplots(1, figsize=(12, 10))
    # cs=ax.pcolormesh(np.array(xgo_phi).reshape(12,6),np.array(ygo_phi).reshape(12,6),param_matrix_phi[:,0,0].reshape(12,6),   cmap=cmap_temp, shading="flat")
    cs,ax=fugi.plot_interpolation(np.array(xgo_phi).reshape(12,6),np.array(ygo_phi).reshape(12,6),param_matrix_phi[:,0,0].reshape(12,6) *100, ax=ax, cmap=cmap_temp)
    
    ax.set_title("R CR-CL b1 map "+ID+" "+CH+" "+en)
    aspect=(np.array(xgo_phi).max()/np.array(ygo_phi).max())
    ax.set_xlabel('\u03C6$_{ph}$ [DEG]')
    ax.set_ylabel('cos\u03D1$_{ph}$ [adm]')
    ax.set_aspect(aspect)
    
    # cbar = fugi.clippedcolorbar(cs,shrink=0.9, ax=cs)
    cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
    # cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
    cbar.ax.set_ylabel('% of asymmetry')
    
    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/b1map_R-S_phi_smooth_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
if fileSCL and fileSCR is not None:
    # fugi = reload(fugi)
    fig,axes= plt.subplots(6,1, figsize=(10, 40), sharex=True)
    fig.suptitle("b1 map "+en)
    
    for ax,i in zip(axes,range(6)):
        # cs,ax=fugi.plot_interpolation(xgo_uniq,ygo_uniq,param_matrix_phi[:,i,0].reshape(len(xgo_uniq),-1).T, ax=ax, cmap=cmap_temp)
        cs,ax=fugi.plot_interpolation(np.array(xgo_phi).reshape(12,6),np.array(ygo_phi).reshape(12,6),param_matrix_phi[:,i,0].reshape(12,6), ax=ax, cmap=cmap_temp)
        # cs,ax=fugi.plot_interpolation(xgo_cos,ygo_cos,param_matrix_cos[:,0,0].T*100, ax=ax, cmap=cmap_temp)

        aspect=(np.array(xgo_phi).max()/np.array(ygo_phi).max())
        ax.set_ylabel('cos\u03D1$_{ph}$ [adm]')
        cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
        cbar.ax.set_ylabel("asymmetry")
        if i==5:
            ax.set_xlabel('\u03C6$_{ph}$ [DEG]')
    fig.tight_layout()

In [ ]:
if fileSCL is None:
    #iminuit with error: NORMALIZED DATA
    from scipy.interpolate import make_interp_spline, BSpline
    import functions_minuit as fumi
    #integrated in the function fumi.plotminuit
    # limits=[(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
    # limits=[(-1, 1), (-1, 2), (None, None), (None, None), (None, None), (None, None)]
    limits=[(None, None), (None, None), (None, None), (None, None), (None, None), (None, None)]

    m=fumi.iminuit_err(cosM,cPECD_norm_phi_sm[0],errPECD_phi[0],limits)
    print(m)

    x_data=cosM;y_data=cPECD_norm_phi_sm[0]

    x_data_new = np.linspace(x_data.min(), x_data.max(), 300)
    bspl = make_interp_spline(x_data, fumi.PECD6(x_data, *m.values), k=5)

    plt.plot(x_data_new, bspl(x_data_new))
    plt.scatter(cosM,cPECD_norm_phi_sm[0],color='r')
    plt.errorbar(cosM,cPECD_norm_phi_sm[0], yerr=errPECD_phi[0], fmt="o")
    plt.xlabel('cos\u03D1$_{ph}$')
    plt.ylabel('PECD')

    plt.show()
    for p in m.parameters:
        print("{} = {:2.3e} +/- {:2.3e}".format(p, m.values[p], m.errors[p]))

In [ ]:
if fileSCL is None:
    #R phi ordered modified with error
    # param_matrix=np.zeros((6,72)) # alternative nparray initialization
    param_matrix_phi = [[0 for i in range(6)] for j in range(72)] #i col, j row
    
    # fig, axes = plt.subplots(6,12, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
    fig, axes = plt.subplots(6,12, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
    custom_xlim = (-1, 1)
    custom_ylim = (-0.2,0.2) #custom_ylim = (-1,1)
    
    # limits=[(-1, 1), (-1, 2), (None, None), (-1, 2), (None, None), (-1, 2)]
    limits=[(-1, 1), (-1, 2), (None, None), (None, None), (None, None), (None, None)]
    
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    # for i,ax in enumerate(axes.ravel()): #for cos
    
        x_data = cosM; y_data = cPECD_norm_phi_sm[i];y_err = errPECD_phi[i]
        
        m=fumi.iminuit_err(x_data,y_data,y_err,limits)
        fumi.plotminuit(x_data,y_data,y_err,m,ax,custom_xlim,custom_ylim)
    
        for counter,el in enumerate(m.parameters): # len = 6
            param_matrix_phi[i][counter]=((m.values[el], m.errors[el]))
    
    param_matrix_phi=np.array(param_matrix_phi) #numpy tranformation!!!
    
    fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)
    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/b1matrix_RCLCR_phi_smooth_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')
        # for p in m.parameters:
        #     print("{} = {:2.3e} +/- {:2.3e}".format(p, m.values[p], m.errors[p]))

In [ ]:
if fileSCL is None:
    fig,ax= plt.subplots(1, figsize=(12, 10))
    # cs=ax.pcolormesh(np.array(xgo_phi).reshape(12,6),np.array(ygo_phi).reshape(12,6),param_matrix_phi[:,0,0].reshape(12,6),   cmap=cmap_temp, shading="flat")
    cs,ax=fugi.plot_interpolation(np.array(xgo_phi).reshape(12,6),np.array(ygo_phi).reshape(12,6),param_matrix_phi[:,0,0].reshape(12,6) *100, ax=ax, cmap=cmap_temp)
    
    ax.set_title("R CR-CL b1 map "+ID+" "+CH+" "+en)
    aspect=(np.array(xgo_phi).max()/np.array(ygo_phi).max())
    ax.set_xlabel('\u03C6$_{ph}$ [DEG]')
    ax.set_ylabel('cos\u03D1$_{ph}$ [adm]')
    ax.set_aspect(aspect)
    
    # cbar = fugi.clippedcolorbar(cs,shrink=0.9, ax=cs)
    cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
    # cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
    cbar.ax.set_ylabel('% of asymmetry')
    
    fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/b1map_RCLCR_phi_smooth_"+CH+"_"+ID+"_"+en+"_exp.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
if fileSCL is None:
    fig,axes= plt.subplots(6,1, figsize=(10, 40), sharex=True)
    fig.suptitle("b1 map "+en)
    
    for ax,i in zip(axes,range(6)):
        # cs,ax=fugi.plot_interpolation(xgo_uniq,ygo_uniq,param_matrix_phi[:,i,0].reshape(len(xgo_uniq),-1).T, ax=ax, cmap=cmap_temp)
        cs,ax=fugi.plot_interpolation(np.array(xgo_phi).reshape(12,6),np.array(ygo_phi).reshape(12,6),param_matrix_phi[:,i,0].reshape(12,6), ax=ax, cmap=cmap_temp)
        # cs,ax=fugi.plot_interpolation(xgo_cos,ygo_cos,param_matrix_cos[:,0,0].T*100, ax=ax, cmap=cmap_temp)
        
        ax.set_aspect(aspect)
        ax.set_ylabel('cos\u03D1$_{ph}$ [adm]')
        cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
        cbar.ax.set_ylabel("asymmetry")
        if i==5:
            ax.set_xlabel('\u03C6$_{ph}$ [DEG]')
    fig.tight_layout()